this file is just to connect to the ebay API to request all the images related to searches. mainly a way to gather more images of certain types of items

In [ ]:
import requests
import os
from urllib.request import urlretrieve

def get_ebay_listing_photos(search_query, app_id, max_results=50):
    """
    Fetch eBay listing photos for a given search query.

    Args:
        search_query (str): The search term to query eBay listings.
        app_id (str): Your eBay App ID (API key).
        max_results (int): Maximum number of results to fetch (default: 50).

    Returns:
        list: A list of image URLs from the eBay listings.
    """
    endpoint = "https://svcs.ebay.com/services/search/FindingService/v1"
    headers = {"X-EBAY-SOA-SECURITY-APPNAME": app_id}
    params = {
        "OPERATION-NAME": "findItemsByKeywords",
        "SERVICE-VERSION": "1.0.0",
        "SECURITY-APPNAME": app_id,
        "RESPONSE-DATA-FORMAT": "JSON",
        "REST-PAYLOAD": "",
        "keywords": search_query,
        "paginationInput.entriesPerPage": max_results,
    }

    response = requests.get(endpoint, headers=headers, params=params)
    if response.status_code != 200:
        raise Exception(f"API request failed with status code {response.status_code}")

    data = response.json()
    image_urls = []

    try:
        items = data["findItemsByKeywordsResponse"][0]["searchResult"][0]["item"]
        for item in items:
            if "galleryURL" in item:
                image_urls.append(item["galleryURL"][0])
    except KeyError:
        print("No items found or unexpected response format.")

    return image_urls


# Example usage
if __name__ == "__main__":
    # Replace 'your-ebay-app-id' with your actual eBay App ID
    app_id = "your-ebay-app-id"
    search_query = "laptop"
    photos = get_ebay_listing_photos(search_query, app_id)

    print(f"Found {len(photos)} photos:")
    for url in photos:
        print(url)

    # Create the 'dataset' folder if it doesn't exist
    os.makedirs("dataset", exist_ok=True)

    # Save each image into the 'dataset' folder
    for idx, url in enumerate(photos):
        try:
            file_path = os.path.join("dataset", f"image_{idx + 1}.jpg")
            urlretrieve(url, file_path)
            print(f"Saved: {file_path}")
        except Exception as e:
            print(f"Failed to save image {idx + 1}: {e}")